In [14]:
import torch
from torch import nn

# create some data using the linear regression formula

weight = 0.7
bias = 0.3

# create range values
start = 0
end = 1
step = 0.02

# create X and y(features and labels)
# without unsqueeze, errors will pop
X = torch.arange(start, end, step).unsqueeze(dim=1) 
y = weight * X + bias


In [16]:
# split the data
train_split = int(0.8 * len(X))
X_train, y_train, X_test, y_test = X[:train_split], y[:train_split], X[train_split:], y[train_split:]

In [17]:
len(X_train), len(y_train), len(X_test)

(40, 40, 10)